# **Fuite de données**

## **1- Définition**

---



**La fuite de données** (en anglais **Data Leakage**) dans le contexte de mise en place de modèle prédictif est considérée comme l'une des principaux problèmes lors des projets de Data Science.

**Une fuite de données** signifie que le modèle "triche" en recherchant les réponses pendant la phase d'entraînement. Concrétement : des informations sur **la cible** que nous essayons de prédire sont introduites et découvertes pendant l'entraînement. Ces informations ne devraient pas être disponibles lors de l'entraînement et cela permet au modèle d'apprendre ou savoir quelque chose qu'il ne devrait pas savoir autrement.

Ce phénomène est difficile à détecter et se passe de manière imperceptible. Il se produit souvent par accident ou inadvertance lors de la phase de préparation des données.

**La fuite de données** peut aboutir à des modèles trop performant qui fonctionnent avec une grande précision sur les données de test mais qui ont une faible capacité de généralisation lorsqu'ils sont déployés en production.


L'exemple le plus trivial de fuite de données est lorsqu'un modèle utilise la variable cible elle-même comme prédicteur, par exemple pour prédire **qu'il pleut les jours de pluie!**


# **Fuite de données dans les challenges et compétitions de Machine Learning**

---



Des problèmes de Data leakage ont été détectés dans de nombreuses compétitions et challenges de machine learning:

* Lors du concours de Data Mining KDDCup 2007 (KDD signifie Knowledge Discovery in Databases), il y avait deux challenges à partir d'un même jeu de données portant sur les critiques de films Netflix. Le premier challenge était de prédire si chaque utilisateur donnerait un avis pour différents films en 2006, compte tenu des données disponibles jusqu'en 2005. Le deuxième défi, "Combien Avis", était de prédire le nombre d'avis que chaque film allait recevoir en 2006, en utilisant également les données fournies jusqu'en 2005. Pour le premier challenge, un ensemble de données tests avec des données réelles de 2006 a été fourni, ce qui était la principale source de fuite.

Deux concours d'exploration de données médicales ont eu lieu l'année suivante et qui présentait également des problèmes de fuite de données. 

* La coupe KDD de l'année 2008 portait sur la détection du cancer à partir des données de mammographie. En analysant les données de ce concours, des chercheurs ont identifié que l'attribut «ID patient» (ignoré par la plupart des concurrents) avait un pouvoir prédictif énorme et inattendu. Ils émettent l'hypothèse que plusieurs études cliniques, institutions ou autres sources ont été utilisées pour entrainer les modèles et que certaines de ces sources détenaient une connaissance de l'état du patient. Les fuites ont ainsi été facilitées en attribuant des identifiants de patient identiques pour les données de chaque source. La manipulation a ainsi été effectuée sans obscurcir la source.

* Le Challenge INFORMS de 2008 portait sur le problème de diagnostic de pneumonie basé sur les informations du patient de l'hôpital. La cible était à l'origine intégrée en tant que valeur spéciale de une ou plusieurs caractéristiques dans les données fournies aux concurrents. Les organisateurs ont supprimé ces valeurs, mais il a été possible d'identifier des traces de ce retrait des données, ce qui a constitué la source de fuite dans cet exemple.




## **3- Sources de fuite de données**

---



La fuite de données a deux sources principales: 

1.   Les features
2.   Les données d'entrainement



### **3-1 Fuite des features** 

Ce type de fuite de données est souvent appelé **le piège de la machine à remonter le temps** (en anglais **the time machine trap**), et pour une bonne raison: le modèle reçoit des informations du futur sur sa variable cible qui ne devraient pas être disponibles au moment de faire des prédictions lorsque le modèle est déployé.








**Exemple:**
Prenons un exemple d'un ensemble de données médicales pour prédire le cancer. Supposons que nous ayons un attribut booléen pour indiquer si un patient suit un traitement de chimiothérapie ou non. En fait, c'est une feature qui fuit car elle contient une information qui appartient au futur, ce qui signifie qu'elle a été enregistrée après que la variable cible s'est réellement produite (est-ce que le patient a un cancer ou non ?). Naturellement, la grande majorité des personnes atteintes de cancer suivent un traitement de chimiothérapie.

**Astuce:** Pour éviter ce piège, posez-vous quelques questions sur l'ensemble de données : s'agit-il d'un ensemble de données chronologiques (en anglais **time series data**) ou les attributs ont-ils été enregistrés à des moments différents par rapport à la variable cible ? 
Si oui, assurez-vous de supprimer tout attribut qui a été enregistré après l'enregistrement de la variable cible.

### **3-2 Fuite dans les données d'entrainement**

Ce type de fuite de données se produit lorsque le jeu de données d'entrainement est contaminé par le jeu de données de test, soit en ayant des échantillons communs, soit si certaines informations du jeu de données de test ont été divulguées lors de l'étape de préparation des données

**Exemple pratique:**


Une façon possible d'introduire une fuite dans vos données d'entraînement est quand vous essayez de traiter le problème des valeurs manquantes. Tenter de remplir les valeurs manquantes avant le **train-test split** entraînera une fuite de données car l'ensemble de test sera utilisé pour remplir les valeurs manquantes dans l'ensemble d'entrainement.

Nous allons démontrer cette fuite avec le célèbre jeu de données **Titanic**.  Nous allons utiliser **une méthode d'imputation** simple où nous remplaçons les valeurs manquantes par une variable aléatoire d'une plage de valeurs délimitée par deux valeurs : (**mean-std**) et (**mean + std**), où mean et std sont respectivement la moyenne et l'écart de l'attribut manquant :

In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')


Mounted at /content/MyDrive


In [ ]:
cd MyDrive/


/content/MyDrive/MyDrive


In [ ]:
cd "Colab Notebooks"

/content/MyDrive/MyDrive/Colab Notebooks


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [ ]:
titanic_dataset_train = pd.read_csv("titanic_train.csv")
titanic_dataset_test = pd.read_csv("titanic_test.csv")
# full data
titanic_dataset = pd.concat([titanic_dataset_train,titanic_dataset_test],axis=0)
# shuffle the data

In [ ]:
# percentage of missing values
total = titanic_dataset.isnull().sum().sort_values(ascending=False)
percent_1 = titanic_dataset.isnull().sum()/titanic_dataset.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

,Total,%
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2
Fare,0,0.0
Ticket,0,0.0


In [ ]:
# converting the type of "Fare" from float64 to int64
titanic_dataset['Fare'] = titanic_dataset['Fare'].astype(int)

In [ ]:
titanic_dataset['Fare'] = titanic_dataset['Fare'].astype(int)

In [ ]:
# converting the sex attribute to numerical
cat_sex = {"Sex": {"male":1,"female":0}}
titanic_dataset.replace(cat_sex, inplace=True)
titanic_dataset.head(5)

,PassengerId,Survived,Pclass,Name,FullName,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,299,1,1,Saalfeld,Mr. Adolphe,1,NaN,0,0,19988,30,C106,S
1,300,1,1,Baxter,Mrs. James (Helene DeLaudeniere Chaput),0,50.0,0,1,PC 17558,247,B58 B60,C
2,301,1,3,Kelly,Miss. Anna Katherine Annie Kate,0,NaN,0,0,9234,7,NaN,Q
3,302,1,3,McCoy,Mr. Bernard,1,NaN,2,0,367226,23,NaN,Q
4,303,0,3,Johnson,Mr. William Cahoone Jr,1,19.0,0,0,LINE,0,NaN,S


In [ ]:
# converting the child attribute to numerical
titanic_dataset["Child"] = np.where(titanic_dataset['Age'] < 18 , "child" , "adult")
cat_child = {"Child": {"adult":1,"child":0}}
titanic_dataset.replace(cat_child, inplace=True)
titanic_dataset.head(5)

,PassengerId,Survived,Pclass,Name,FullName,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
0,299,1,1,Saalfeld,Mr. Adolphe,1,NaN,0,0,19988,30,C106,S,1
1,300,1,1,Baxter,Mrs. James (Helene DeLaudeniere Chaput),0,50.0,0,1,PC 17558,247,B58 B60,C,1
2,301,1,3,Kelly,Miss. Anna Katherine Annie Kate,0,NaN,0,0,9234,7,NaN,Q,1
3,302,1,3,McCoy,Mr. Bernard,1,NaN,2,0,367226,23,NaN,Q,1
4,303,0,3,Johnson,Mr. William Cahoone Jr,1,19.0,0,0,LINE,0,NaN,S,1


In [ ]:
# there is one missing value in each set, so we can just fill it with the most common value "S"
titanic_dataset["Embarked"]=titanic_dataset["Embarked"].fillna("S")

In [ ]:
# there are 3 unique possible values : S , C , Q
cat_emb = {"Embarked": {"S":0,"Q":1,"C":2}}
titanic_dataset.replace(cat_emb, inplace=True)
titanic_dataset.head(5)

,PassengerId,Survived,Pclass,Name,FullName,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
0,299,1,1,Saalfeld,Mr. Adolphe,1,NaN,0,0,19988,30,C106,0,1
1,300,1,1,Baxter,Mrs. James (Helene DeLaudeniere Chaput),0,50.0,0,1,PC 17558,247,B58 B60,2,1
2,301,1,3,Kelly,Miss. Anna Katherine Annie Kate,0,NaN,0,0,9234,7,NaN,1,1
3,302,1,3,McCoy,Mr. Bernard,1,NaN,2,0,367226,23,NaN,1,1
4,303,0,3,Johnson,Mr. William Cahoone Jr,1,19.0,0,0,LINE,0,NaN,0,1


In [ ]:
# drop unnanessary attributes
attributes_to_drop =["PassengerId","Cabin","Survived","Name","FullName","Ticket"]
#  data
X_titanic_dataset = titanic_dataset.drop(attributes_to_drop,axis=1)
#  labels
Y_titanic_dataset = titanic_dataset["Survived"]


**Scenario 1 :** remplir les valeurs manquantes avant le train-test split

In [ ]:
# dealing with missing values for the "age" attribute
# filling missing values for the age attribute

mean = X_titanic_dataset["Age"].mean()
std = X_titanic_dataset["Age"].std()
is_null = X_titanic_dataset["Age"].isnull().sum()
# compute random numbers between the mean, std and is_null
rand_age = np.random.randint(mean - std, mean + std, size = is_null)
# fill NaN values in Age column with random values generated
age_slice = X_titanic_dataset["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
X_titanic_dataset["Age"] = age_slice
X_titanic_dataset["Age"] = X_titanic_dataset["Age"].astype(int)
X_titanic_dataset["Age"].isnull().sum()


0

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(
X_titanic_dataset, Y_titanic_dataset, test_size=0.33, random_state=42)


In [ ]:
# initilize a guassian naive bayes classifier
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
# probabilities of each sample in the dataset for each label (survived,dead)
gaussian.predict_proba(X_train)

array([[0.97397463, 0.02602537],
       [0.36155316, 0.63844684],
       [0.64951747, 0.35048253],
       ...,
       [0.97339778, 0.02660222],
       [0.94276918, 0.05723082],
       [0.97368316, 0.02631684]])

In [ ]:
# model accuracy
titanic_accuracy = gaussian.score(X_test, y_test) * 100
print("prediction accuracy : %3.3f percent" %titanic_accuracy)

prediction accuracy : 81.356 percent


**Scenario 2 :** remplir les valeurs manquantes après le train-test split

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(
X_titanic_dataset, Y_titanic_dataset, test_size=0.33, random_state=42)

In [ ]:
# dealing with missing values for the "age" attribute
# filling missing values for the age attribute

# filling in missing data in the train set
mean = X_train["Age"].mean()
std = X_train["Age"].std()
is_null = X_train["Age"].isnull().sum()
# compute random numbers between the mean, std and is_null
rand_age = np.random.randint(mean - std, mean + std, size = is_null)
# fill NaN values in Age column with random values generated
age_slice = X_train["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
X_train["Age"] = age_slice
X_train["Age"] = X_train["Age"].astype(int)
X_train["Age"].isnull().sum()


# filling in missing data in the test set
mean = X_test["Age"].mean()
std = X_test["Age"].std()
is_null = X_test["Age"].isnull().sum()
# compute random numbers between the mean, std and is_null
rand_age = np.random.randint(mean - std, mean + std, size = is_null)
# fill NaN values in Age column with random values generated
age_slice = X_test["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
X_test["Age"] = age_slice
X_test["Age"] = X_test["Age"].astype(int)
X_test["Age"].isnull().sum()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

0

In [ ]:
# applying naiveBayes classifier
gaussian = GaussianNB()
gaussian.fit(X_train, y_train)
# probabilities of each sample in the dataset for each label (survived,dead)
gaussian.predict_proba(X_train)

array([[0.9733319 , 0.0266681 ],
       [0.35471667, 0.64528333],
       [0.65468461, 0.34531539],
       ...,
       [0.97322305, 0.02677695],
       [0.94222556, 0.05777444],
       [0.97403942, 0.02596058]])

In [ ]:
# model accuracy
titanic_accuracy = gaussian.score(X_test, y_test) * 100
print("prediction accuracy : %3.3f percent" %titanic_accuracy)

prediction accuracy : 81.695 percent


**Conclusion :**

Nous pouvons voir la différence entre les performances du modèle dans les deux scénarios (81.356 vs. 81.695%), sachant que nous avons gardé le même jeu de données et le même modèle. 

Cela peut sembler une très petite différence, mais rappelez-vous que nous avons utilisé une stratégie d'imputation simple pour une seul variable ("Age"). 

Supposons que nous avons plusieurs variables à imputer maintenant. Et si nous utilisions des méthodes d'imputation plus complexes ? Ou si l'ensemble de données est très grand ? Cela aggraverait le problème.

## **4- Comment éviter et résoudre le problème de Data Leakage**

---



Pour faire face aux fuites de données, vous devez prendre du recul. 
Vous devez obtenir accés aux données brutes et essayer de supprimer les features et les échantillons qui fuient. 

Si vous n'avez pas accès aux données sources d'origine et que vous vous retrouvez avec un ensemble de données, suivez ces astuces et techniques pour minimiser l'impact des fuites de données :



*   L'analyse exploratoire des données (EDA) peut être un outil puissant pour identifier les fuites de données. EDA vous permet de vous familiariser avec les données en les examinant à travers des outils statistiques et de visualisation

*   Supprimez toutes les données postérieur à l'événement qui vous intéresse (la variable cible), en vous concentrant sur le moment où vous avez appris un fait ou une observation plutôt que sur l'heure à laquelle l'observation s'est produite.

*   Ajoutez du bruit aléatoire aux données d'entrée du modèle pour essayer de lisser les effets d'éventuelles fuites des features.

*   Si vous pensez qu'une variable fuit, envisagez de la supprimer.


*   Si la performance de votre modèle est trop bonne pour être vraie, une fuite de données peut en être la cause. Vous devez mettre votre performance au regard des résultats documentés antérieurs ou concurrents pour le problème en question. Une divergence substantielle par rapport à ces performances attendues mérite de tester le modèle de plus près pour s'assurer qu'il n'y a pas de fuite.








**Réference principal** 


*   [Leakage in Data Mining: Formulation, Detection, and Avoidance](https://www.cs.umb.edu/~ding/history/470_670_fall_2011/papers/cs670_Tran_PreferredPaper_LeakingInDataMining.pdf)


